In [1]:
import pandas as pd
import numpy as np

def generate_labels():
    print("⚙️ Cargando dataset crudo...")
    try:
        df = pd.read_csv("nhanes_raw_data.csv")
    except FileNotFoundError:
        print("❌ Error: No encuentro 'nhanes_raw_data.csv'.")
        return

    initial_count = len(df)
    print(f"📄 Filas iniciales: {initial_count}")

    # ==========================================
    # 1. SANITY CHECKS (Hard Rules) - Mandamiento #5
    # ==========================================
    # Eliminamos valores fisiológicamente imposibles (Artefactos/Errores de lab)
    # Rangos extremos definidos para seguridad:
    mask_impossible = (
        (df['HGB'] > 25) | (df['HGB'] < 2) |    # Hemoglobina imposible
        (df['WBC'] > 200) | (df['WBC'] < 0.1) | # Leucocitos incompatibles con la vida (ambulatoria)
        (df['PLT'] > 2000) | (df['PLT'] < 0) |  # Plaquetas error
        (df['Age'] > 120)                       # Edad error
    )
    
    df_clean = df[~mask_impossible].copy()
    dropped = initial_count - len(df_clean)
    if dropped > 0:
        print(f"🧹 Sanity Check: Se eliminaron {dropped} filas con valores imposibles.")

    # ==========================================
    # 2. TARGET ENGINEERING (Crear la columna 'Diagnosis')
    # ==========================================
    # Definimos condiciones vectorizadas (NO bucles for)
    
    # Condición de Infección (Leucocitosis)
    cond_infection = df_clean['WBC'] > 11.0
    
    # Condición de Trombocitopenia
    cond_thrombo = df_clean['PLT'] < 150.0
    
    # Condición de Anemia (Depende del Sexo: 1=Hombre, 2=Mujer)
    cond_anemia = (
        ((df_clean['Sex'] == 1) & (df_clean['HGB'] < 13.0)) |
        ((df_clean['Sex'] == 2) & (df_clean['HGB'] < 12.0))
    )
    
    # Jerarquía de Diagnóstico (Si tiene varias, cuál gana?)
    # Prioridad: Infección > Trombocitopenia > Anemia > Normal
    # (Esto simplifica el problema a Multi-Clase Single Label para empezar)
    
    conditions = [cond_infection, cond_thrombo, cond_anemia]
    choices = ['Infection', 'Thrombocytopenia', 'Anemia']
    
    # np.select evalúa en orden. Si no cumple ninguna, pone 'Normal'
    df_clean['Diagnosis'] = np.select(conditions, choices, default='Normal')
    
    # ==========================================
    # 3. REPORTE DE BALANCE DE CLASES
    # ==========================================
    print("\n📊 Distribución de Clases Generada (Target):")
    print(df_clean['Diagnosis'].value_counts())
    print(f"\n% de Distribución:\n{df_clean['Diagnosis'].value_counts(normalize=True) * 100}")

    # Guardar dataset maestro listo para ML
    output_file = "nhanes_labeled_data.csv"
    df_clean.to_csv(output_file, index=False)
    print(f"\n💾 Guardado exitosamente: {output_file}")

if __name__ == "__main__":
    generate_labels()

⚙️ Cargando dataset crudo...
📄 Filas iniciales: 7528
🧹 Sanity Check: Se eliminaron 1 filas con valores imposibles.

📊 Distribución de Clases Generada (Target):
Diagnosis
Normal              5792
Anemia              1060
Infection            440
Thrombocytopenia     235
Name: count, dtype: int64

% de Distribución:
Diagnosis
Normal              76.949648
Anemia              14.082636
Infection            5.845622
Thrombocytopenia     3.122094
Name: proportion, dtype: float64

💾 Guardado exitosamente: nhanes_labeled_data.csv
